In [1]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# task a

In [ ]:
df_sales = pd.read_csv("/content/Alcohol_Sales.csv")
df_sales.head()

,DATE,S4248SM144NCEN
0,1992-01-01,3459
1,1992-02-01,3458
2,1992-03-01,4002
3,1992-04-01,4564
4,1992-05-01,4221


In [ ]:
df_sales.shape

(325, 2)

In [ ]:
df_sales.columns = ["DATE", "SALES"]

In [ ]:
df_sales.head()

,DATE,SALES
0,1992-01-01,3459
1,1992-02-01,3458
2,1992-03-01,4002
3,1992-04-01,4564
4,1992-05-01,4221


In [ ]:
df_sales.dtypes

DATE     object
SALES     int64
dtype: object

In [ ]:
df_sales['DATE'] = pd.to_datetime(df_sales['DATE'])
df_sales.set_index('DATE', inplace=True)

In [ ]:
train_data, test_data = train_test_split(df_sales, test_size=0.2, shuffle=False)


In [ ]:
scaler = StandardScaler()
train_data['SALES'] = scaler.fit_transform(train_data[['SALES']])
test_data['SALES'] = scaler.transform(test_data[['SALES']])


In [ ]:
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        seq = data.iloc[i:i + sequence_length]
        target = data.iloc[i + sequence_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

sequence_length = 10  # Adjust this value as needed
X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(1))  # Output layer for forecasting

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
8/8 [==============================] - 2s 52ms/step - loss: 0.8510 - val_loss: 4.8219
Epoch 2/100
8/8 [==============================] - 0s 13ms/step - loss: 0.6357 - val_loss: 3.2489
Epoch 3/100
8/8 [==============================] - 0s 15ms/step - loss: 0.4092 - val_loss: 1.3042
Epoch 4/100
8/8 [==============================] - 0s 13ms/step - loss: 0.2071 - val_loss: 3.4554
Epoch 5/100
8/8 [==============================] - 0s 14ms/step - loss: 0.1948 - val_loss: 1.9476
Epoch 6/100
8/8 [==============================] - 0s 14ms/step - loss: 0.1734 - val_loss: 0.5668
Epoch 7/100
8/8 [==============================] - 0s 11ms/step - loss: 0.1728 - val_loss: 0.6402
Epoch 8/100
8/8 [==============================] - 0s 16ms/step - loss: 0.1617 - val_loss: 1.1642
Epoch 9/100
8/8 [==============================] - 0s 15ms/step - loss: 0.1651 - val_loss: 1.0874
Epoch 10/100
8/8 [==============================] - 0s 14ms/step - loss: 0.1626 - val_loss: 0.8392
Epoch 11/100
8/8 [=

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error (RMSE): {rmse}')


2/2 [==============================] - 0s 7ms/step
Root Mean Squared Error (RMSE): 1.1419441638625467


# task b

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

2023-09-20 19:35:21.657708: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 19:35:26.692193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [38]:
df_miles = pd.read_csv("data/task4/Miles_Traveled.csv")

In [39]:
df_miles.head(3)

,DATE,TRFVOLUSM227NFWA
0,1970-01-01,80173.0
1,1970-02-01,77442.0
2,1970-03-01,90223.0


In [40]:
df_miles.columns = ["DATE", "MILES"]

In [41]:
# current date format is yyyy-mm-dd
df_miles.index = pd.to_datetime(df_miles["DATE"], format = "%Y-%m-%d")
# df_miles.index = df_miles.index.strftime('%d-%m-%Y')

In [42]:
df_miles['day'] = df_miles.index.day
df_miles['month'] = df_miles.index.month
df_miles['year'] = df_miles.index.year

In [56]:
df_miles = df_miles.sort_values(by=['year', 'day'])

In [36]:
#df_miles = df_miles.drop(columns=['day', 'month', 'year'])

In [57]:
df_miles[:20]

,DATE,MILES,day,month,year
DATE,,,,,
1970-01-01,1970-01-01,80173.0,1,1,1970
1970-12-01,1970-12-01,89911.0,1,12,1970
1970-11-01,1970-11-01,89684.0,1,11,1970
1970-10-01,1970-10-01,96317.0,1,10,1970
1970-08-01,1970-08-01,106969.0,1,8,1970
1970-07-01,1970-07-01,106392.0,1,7,1970
1970-09-01,1970-09-01,95254.0,1,9,1970
1970-05-01,1970-05-01,97972.0,1,5,1970
1970-04-01,1970-04-01,89956.0,1,4,1970
